In [ ]:
import momepy
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, box
from shapely.ops import unary_union
import planetary_computer
import pystac_client
import dask.dataframe
import dask_geopandas as dgd
import dask.distributed
import deltalake
import shapely.geometry
import mercantile
import rasterio
from rasterio.windows import from_bounds
from pyproj import transform
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterio.mask
from rasterio.plot import show
from shapely.geometry import shape, Polygon, mapping
import fiona
from rasterio.features import shapes

In [ ]:
local_crs = 27700
place = "glasgow"
latlng = (-4.251846930489373, 55.86421405612109)
dist = 30000
country = "united kingdom"
crs=4326

In [ ]:
def create_square(center_point, distance):
    """
    Create a square polygon centered on the given point.
    The 'distance' parameter is half the side length of the square.
    """
    x, y = center_point.x, center_point.y
    return Polygon([(x - distance, y - distance),
                    (x - distance, y + distance),
                    (x + distance, y + distance),
                    (x + distance, y - distance)])

# Create a GeoSeries with the specified point
gdf = gpd.GeoSeries([Point(latlng[0], latlng[1])], crs=crs)

# Reproject to a coordinate system that uses meters (UTM)
gdf_utm = gdf.to_crs(epsg=local_crs)

# Get the center point in UTM coordinates
center_point = gdf_utm[0]

# Create a square polygon centered on the point
# The distance parameter is half the side length of the square.
# For a 50 km square, the distance will be 25 km.
square = create_square(center_point, 25000)  # 25 km half the side length

# Create a GeoSeries for the square in UTM coordinates
square_gs_utm = gpd.GeoSeries([square], crs=gdf_utm.crs)

# Reproject the square back to the original CRS
area_of_interest = square_gs_utm.to_crs(epsg=crs)

# Plotting
area_of_interest.explore()

In [ ]:
# # Create a GeoDataFrame with the specified point
# gdf = gpd.GeoDataFrame(geometry=[Point(latlng[0], latlng[1])], crs=crs)

# # Reproject to a coordinate system that uses meters (UTM)
# gdf_utm = gdf.to_crs(epsg=local_crs)

# # Create a 50 km buffer around the point
# buffer = gdf_utm.buffer(10000)  # 50 km buffer

# # Reproject buffer back to original CRS
# area_of_interest = buffer.to_crs(epsg=crs)

# # Plottin
# area_of_interest.explore()



In [ ]:
type(area_of_interest)

In [ ]:
# Read only the portion of the GeoTIFF that intersects with the AOI

cell_polygons = []
with rasterio.open("output/built_height.tif") as src:
    # Calculate the window to read based on AOI bounds
    out_image, transformed = rasterio.mask.mask(src, area_of_interest, crop=True, filled=True)
    out_profile = src.profile.copy()
    
out_profile.update({'width': out_image.shape[2],'height': out_image.shape[1], 'transform': transformed})
with rasterio.open(f"output/{place}.tif", 'w', **out_profile) as dst:
    dst.write(out_image)
    
with rasterio.open(f"output/{place}.tif") as src:
    show(src)

In [ ]:
def get_cell_polygon(x, y, transform):
    """
    Create a polygon for the given cell coordinates (x, y) using the affine transform.
    """
    tl = transform * (x, y)
    tr = transform * (x + 1, y)
    br = transform * (x + 1, y + 1)
    bl = transform * (x, y + 1)
    return Polygon([tl, tr, br, bl, tl])

In [ ]:
mask = None
with rasterio.Env():
    with rasterio.open(f"output/{place}.tif") as src:
        image = src.read(1) # first band
        transform = src.transform

        # Prepare schema for shapefile
        schema = {
            'properties': [('raster_val', 'int')],
            'geometry': 'Polygon'
        }

        with fiona.open(f"output/{place}_height_cells.shp", 'w', 
                        driver='ESRI Shapefile',
                        crs=src.crs,
                        schema=schema) as dst:
            
            # Iterate over each pixel in the raster
            for row in range(image.shape[0]):
                for col in range(image.shape[1]):
                    value = image[row, col]
                    polygon = get_cell_polygon(col, row, transform)
                    dst.write({
                        'properties': {'raster_val': int(value)},
                        'geometry': mapping(polygon)
                    })

In [ ]:
area_of_interest[0].bounds

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
collection = catalog.get_collection("ms-buildings")

asset = collection.assets["delta"]

storage_options = {
    "account_name": asset.extra_fields["table:storage_options"]["account_name"],
    "sas_token": asset.extra_fields["table:storage_options"]["credential"],
}
table = deltalake.DeltaTable(asset.href, storage_options=storage_options)

quadkeys = [
    int(mercantile.quadkey(tile))
    for tile in mercantile.tiles(*area_of_interest[0].bounds, zooms=9)
]
quadkeys

uris = table.file_uris([("quadkey", "in", quadkeys)])
uris

df = dgd.read_parquet(uris, storage_options=storage_options)


In [ ]:
buildings = dgd.sjoin(df, gpd.GeoDataFrame(geometry=area_of_interest), how="inner", op="intersects").compute()
buildings.explore()

In [20]:
buildings.to_parquet('output/building_footprints/glasgow_buildings_50km.pq', index=False)
# buildings = gpd.read_parquet('output/building_footprints/glasgow_buildings_50km.pq')